In [1]:
from pyspark.sql import SparkSession
import mysql.connector

In [2]:
# Spark 세션 시작
spark = SparkSession.builder \
    .appName("CSV to MariaDB") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/24 05:12:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/24 05:12:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/24 05:12:50 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/09/24 05:12:50 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
connection = mysql.connector.connect(
    host="13.124.90.34",
    user="root",
    password="enqnrhkwk108",
    database="S11P21D108",
    charset="utf8mb4",  # 문자셋 설정
    collation="utf8mb4_general_ci",  # collation 설정
    autocommit=False,  # Auto-commit을 끄고 트랜잭션 처리
    connection_timeout=28800,  # 타임아웃 증가
)

In [4]:
# CSV 파일 읽기
store_df = spark.read.option("encoding", "cp949").csv("hdfs://master:9000/input/서울시_상권분석서비스(점포-상권).csv", header=True, inferSchema=True)


In [5]:
# CSV 파일 읽기
gu_df = spark.read.option("encoding", "cp949").csv("hdfs://master:9000/input/서울시_상권분석서비스(점포-자치구).csv", header=True, inferSchema=True)


In [6]:
# CSV 파일 읽기
dong_df = spark.read.option("encoding", "cp949").csv("hdfs://master:9000/input/서울시_상권분석서비스(점포-행정동).csv", header=True, inferSchema=True)


In [7]:
# 필요한 컬럼 선택
store_df = store_df.select('기준_년분기_코드', '상권_코드', '서비스_업종_코드',
                     '서비스_업종_코드_명', '점포_수', '유사_업종_점포_수', '개업_율', '개업_점포_수', 
                     '폐업_률', '폐업_점포_수', '프랜차이즈_점포_수')

# 기준_년분기_코드가 20241인 데이터 필터링
store_2024 = store_df.filter(store_df['기준_년분기_코드'] == 20241)

# 결과 확인
store_2024.show()

+----------------+---------+----------------+-------------------+-------+-----------------+-------+------------+-------+------------+------------------+
|기준_년분기_코드|상권_코드|서비스_업종_코드|서비스_업종_코드_명|점포_수|유사_업종_점포_수|개업_율|개업_점포_수|폐업_률|폐업_점포_수|프랜차이즈_점포_수|
+----------------+---------+----------------+-------------------+-------+-----------------+-------+------------+-------+------------+------------------+
|           20241|  3001491|        CS100001|         한식음식점|    171|              176|      3|           6|      1|           2|                 5|
|           20241|  3001491|        CS100002|         중식음식점|     18|               18|      0|           0|      0|           0|                 0|
|           20241|  3001491|        CS100003|         일식음식점|     23|               24|      0|           0|      0|           0|                 1|
|           20241|  3001491|        CS100004|         양식음식점|    220|              223|      2|           4|      2|           4|                 3|
|        

In [8]:
# 필요한 컬럼 선택
gu_df = gu_df.select('기준_년분기_코드', '자치구_코드', '서비스_업종_코드',
                     '서비스_업종_코드_명', '점포_수', '유사_업종_점포_수', '개업_율', '개업_점포_수', 
                     '폐업_률', '폐업_점포_수', '프랜차이즈_점포_수')

# 기준_년분기_코드가 20241인 데이터 필터링
gu_2024 = gu_df.filter(gu_df['기준_년분기_코드'] == 20241)

# 결과 확인
gu_2024.show()

+----------------+-----------+----------------+----------------------+-------+-----------------+-------+------------+-------+------------+------------------+
|기준_년분기_코드|자치구_코드|서비스_업종_코드|   서비스_업종_코드_명|점포_수|유사_업종_점포_수|개업_율|개업_점포_수|폐업_률|폐업_점포_수|프랜차이즈_점포_수|
+----------------+-----------+----------------+----------------------+-------+-----------------+-------+------------+-------+------------+------------------+
|           20241|      11740|        CS300043|          전자상거래업| 2018.0|           2022.0|    0.1|         3.0|    5.8|       117.0|               4.0|
|           20241|      11740|        CS300042|                주유소|   37.0|             37.0|    0.0|         0.0|    2.7|         1.0|               0.0|
|           20241|      11740|        CS300041|                예술품|   31.0|             31.0|    3.2|         1.0|    0.0|         0.0|               0.0|
|           20241|      11740|        CS300040|       재생용품 판매점|   40.0|             40.0|    0.0|         0.0|    2.5|       

In [9]:
# 필요한 컬럼 선택
dong_df = dong_df.select('기준_년분기_코드', '행정동_코드', '서비스_업종_코드',
                     '서비스_업종_코드_명', '점포_수', '유사_업종_점포_수', '개업_율', '개업_점포_수', 
                     '폐업_률', '폐업_점포_수', '프랜차이즈_점포_수')

# 기준_년분기_코드가 20241인 데이터 필터링
dong_2024 = dong_df.filter(dong_df['기준_년분기_코드'] == 20241)

# 결과 확인
dong_2024.show()

+----------------+-----------+----------------+-------------------+-------+-----------------+-------+------------+-------+------------+------------------+
|기준_년분기_코드|행정동_코드|서비스_업종_코드|서비스_업종_코드_명|점포_수|유사_업종_점포_수|개업_율|개업_점포_수|폐업_률|폐업_점포_수|프랜차이즈_점포_수|
+----------------+-----------+----------------+-------------------+-------+-----------------+-------+------------+-------+------------+------------------+
|           20241|   11110515|        CS100001|         한식음식점|     93|               93|      2|           2|      2|           2|                 0|
|           20241|   11110515|        CS100002|         중식음식점|      2|                2|      0|           0|      0|           0|                 0|
|           20241|   11110515|        CS100003|         일식음식점|     15|               16|     13|           2|     13|           2|                 1|
|           20241|   11110515|        CS100004|         양식음식점|     49|               49|      6|           3|      0|           0|                

In [18]:
def s_area(row):
         # 1. service_code가 service 테이블에 존재하는지 확인
        select_service_sql = """
        SELECT service_code FROM service WHERE service_code = %s
        """
        cursor.execute(select_service_sql, (row['서비스_업종_코드'],))
        service_exists = cursor.fetchone()

        # 2. service_code가 존재하지 않으면 service 테이블에 삽입
        if not service_exists:
            insert_service_sql = """
            INSERT INTO service (service_code, service_name) VALUES (%s, %s)
            """
            cursor.execute(insert_service_sql, (row['서비스_업종_코드'], row['서비스_업종_코드_명']))

        sql = """
        INSERT INTO store (service_code, area_id, gu_code, dong_code, year, store_cnt, sim_busin_cnt,
                           opening_rate, closing_rate, opening_cnt, closing_cnt, fran_cnt)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(sql, (row['서비스_업종_코드'], row['상권_코드'],None,None,row['기준_년분기_코드'], row['점포_수'], 
                             row['유사_업종_점포_수'], row['개업_율'], row['개업_점포_수'],
                             row['폐업_률'], row['폐업_점포_수'], row['프랜차이즈_점포_수']))

In [19]:
def s_gu(row):
             # 1. service_code가 service 테이블에 존재하는지 확인
        select_service_sql = """
        SELECT service_code FROM service WHERE service_code = %s
        """
        cursor.execute(select_service_sql, (row['서비스_업종_코드'],))
        service_exists = cursor.fetchone()

        # 2. service_code가 존재하지 않으면 service 테이블에 삽입
        if not service_exists:
            insert_service_sql = """
            INSERT INTO service (service_code, service_name) VALUES (%s, %s)
            """
            cursor.execute(insert_service_sql, (row['서비스_업종_코드'], row['서비스_업종_코드_명']))

        sql = """
        INSERT INTO store (service_code, area_id, gu_code, dong_code, year, store_cnt, sim_busin_cnt,
                           opening_rate, closing_rate, opening_cnt, closing_cnt, fran_cnt)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(sql, (row['서비스_업종_코드'],None, row['자치구_코드'],None,row['기준_년분기_코드'], row['점포_수'], 
                             row['유사_업종_점포_수'], row['개업_율'], row['개업_점포_수'],
                             row['폐업_률'], row['폐업_점포_수'], row['프랜차이즈_점포_수']))

In [20]:
def s_dong(row):
             # 1. service_code가 service 테이블에 존재하는지 확인
        select_service_sql = """
        SELECT service_code FROM service WHERE service_code = %s
        """
        cursor.execute(select_service_sql, (row['서비스_업종_코드'],))
        service_exists = cursor.fetchone()

        # 2. service_code가 존재하지 않으면 service 테이블에 삽입
        if not service_exists:
            insert_service_sql = """
            INSERT INTO service (service_code, service_name) VALUES (%s, %s)
            """
            cursor.execute(insert_service_sql, (row['서비스_업종_코드'], row['서비스_업종_코드_명']))

        sql = """
        INSERT INTO store (service_code, area_id, gu_code, dong_code, year, store_cnt, sim_busin_cnt,
                           opening_rate, closing_rate, opening_cnt, closing_cnt, fran_cnt)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(sql, (row['서비스_업종_코드'],None,None, row['행정동_코드'],row['기준_년분기_코드'], row['점포_수'], 
                             row['유사_업종_점포_수'], row['개업_율'], row['개업_점포_수'],
                             row['폐업_률'], row['폐업_점포_수'], row['프랜차이즈_점포_수']))

In [21]:
cursor = connection.cursor()

# DataFrame의 각 행을 MariaDB에 삽입
for row in store_2024.collect():
    s_area(row)

# DataFrame의 각 행을 MariaDB에 삽입
for row in gu_2024.collect():
    s_gu(row)

# DataFrame의 각 행을 MariaDB에 삽입
for row in dong_2024.collect():
    s_dong(row)

# 변경사항 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()